Let's implement the per-commitment secret generator!

```python
generate_from_seed(seed, I):
    P = seed
    for B in 47 down to 0:
        if B set in I:
            flip(B) in P
            P = SHA256(P)
    return P
```

In [48]:
def flip(bites, bit):
    blist = list(bites)
    flip_which_byte = bit // 8
    flip_which_bit = bit % 8
    blist[flip_which_byte] ^= 1 << flip_which_bit
    return bytes(blist)
    
def to_b(hex):
    return bytes.fromhex(hex)

In [103]:
from hashlib import sha256


def generate_from_seed(seed_bytes, commit_idx):
    P_bytes = seed_bytes
    assert commit_idx < 2 ** 48 and commit_idx > 0
    for bit in range(47, -1, -1):
        if (commit_idx >> bit) % 2 == 1:
            flip(P_bytes, bit)
            P_bytes = sha256(P_bytes).digest()
            
    return P_bytes

secret = generate_from_seed(bytes.fromhex('0101010101010101010101010101010101010101010101010101010101010101'), 2)
print(secret.hex())

72cd6e8422c407fb6d098690f1130b7ded7ec2f7f5e1d30bd9d521f015363793


TODO: make these tests pass

name: generate_from_seed 0 final node
seed: 0x0000000000000000000000000000000000000000000000000000000000000000
I: 281474976710655
output: 0x02a40c85b6f28da08dfdbe0926c53fab2de6d28c10301f8f7c4073d5e42e3148

name: generate_from_seed FF final node
seed: 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
I: 281474976710655
output: 0x7cc854b54e3e0dcdb010d7a3fee464a9687be6e8db3be6854c475621e007a5dc

name: generate_from_seed FF alternate bits 1
seed: 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
I: 0xaaaaaaaaaaa
output: 0x56f4008fb007ca9acf0e15b054d5c9fd12ee06cea347914ddbaed70d1c13a528

name: generate_from_seed FF alternate bits 2
seed: 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
I: 0x555555555555
output: 0x9015daaeb06dba4ccc05b91b2f73bd54405f2be9f217fbacd3c5ac2e62327d31

name: generate_from_seed 01 last nontrivial node
seed: 0x0101010101010101010101010101010101010101010101010101010101010101
I: 1
output: 0x915c75942a26bb3a433a8ce2cb0427c29ec6c1775cfc78328b57f6ba7bfeaa9c

## Writing the Commitment Index to the Transaction

Step 1: what's your commitment index?
    A: 32
    

In [83]:
# Step 2: what's your payment_basepoint?
# A: 666 -> 037f75c66c45a52c35ead5970bbfaafdfba626a6ddceabc14e0f8a8c7d88a5772b
import coincurve

my_payment_basepoint = coincurve.PrivateKey.from_int(666).public_key
my_payment_basepoint.format().hex()
my_payment_basepoint = bytes.fromhex('034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa')

In [85]:
# Step 3: what's peer's payment_basepoint
# A: 777 -> 03e1fe434d345bf33083abb6280f4f44ac5fb22934977813c20c015f2b43d3fab8

peer_payment_basepoint = coincurve.PrivateKey.from_int(777).public_key
peer_payment_basepoint.format().hex()
peer_payment_basepoint = bytes.fromhex('032c0b7cf95324a07d05398b240174dc0c2be444d96b159aa6c7f7b1e668680991')

In [87]:
obscuring_data = sha256(my_payment_basepoint + peer_payment_basepoint).digest()
obscuring_data.hex()

'c8bfea84214b45899482a4bad1d85c42130743ed78ba3711f5532bb038521914'

In [88]:
commitment_index = (42).to_bytes(48 // 8, 'big')
commitment_index.hex()

'00000000002a'

In [90]:
obscure_commit_idx = bytes([ x ^ y for x, y in zip(list(obscuring_data[-6:]), list(commitment_index))])

print(commitment_index.hex(), obscuring_data[-6:].hex(), obscure_commit_idx.hex())

00000000002a 2bb038521914 2bb03852193e


In [91]:
locktime = bytes([0x20]) + obscure_commit_idx[3:]
sequence = bytes([0x80]) + obscure_commit_idx[:3]

In [92]:
print(sequence.hex())
print(locktime.hex())

802bb038
2052193e
